In [1]:
pip install openpyxl==3.0.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5


In [51]:
import openpyxl
from openpyxl.styles import Alignment, Font
from datetime import datetime

# 現在の日付を取得してファイル名を作成
current_date = datetime.now().strftime("%Y%m%d")
file_name = f"請求書_{current_date}.xlsx"

# Excel ワークブックとシートを作成
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "請求書"

# 固定データ（請求先情報）
billing_info = [
    "請求書",
    "",
    "株式会社ABC",
    "〒101-0022 東京都千代田区神田練塀町300",
    "TEL: 03-1234-5678    FAX: 03-1234-5678",
    "担当者名: 鈴木一郎 様",
    "",
]

# 商品データ
products = [
    ["商品名", "数量", "単価", "金額"],
    ["商品A", 2, 10000, 2 * 10000],
    ["商品B", 1, 15000, 1 * 15000],
]

# 消費税率
tax_rate = 0.1

# 1. 請求先情報を記載（2行目2列目から開始）
for i, info in enumerate(billing_info, start=2):
    ws[f"B{i}"] = info  # 列をBに変更
    ws.merge_cells(start_row=i, start_column=2, end_row=i, end_column=5)  # B～E列をマージ
    ws[f"B{i}"].alignment = Alignment(horizontal="left")

# 空行
start_row = len(billing_info) + 2

# 2. 商品情報を記載（請求先情報の下から開始）
for i, product in enumerate(products, start=start_row):
    for j, value in enumerate(product, start=2):  # 列をB（2列目）から開始
        ws.cell(row=i, column=j).value = value
        ws.cell(row=i, column=j).alignment = Alignment(horizontal="center")
        if i == start_row:  # ヘッダー行を太字にする
            ws.cell(row=i, column=j).font = Font(bold=True)

# 合計金額の計算
subtotal = sum(row[3] for row in products[1:])  # 金額列の合計
tax = int(subtotal * tax_rate)  # 消費税
total = subtotal + tax

#合計金額を表示
ws["E12"] = subtotal
ws["E12"].alignment = Alignment(horizontal="center")  # 左よせ


# 3. 合計金額を記載（商品情報の下に配置）
summary_data = [
    ["小計", subtotal],
    ["消費税", tax],
    ["税込合計", total],
]

summary_start_row = start_row + len(products) + 2
for i, (label, value) in enumerate(summary_data, start=summary_start_row):
    ws[f"C{i}"] = label
    ws[f"D{i}"] = value
    ws[f"C{i}"].font = Font(bold=True)
    ws[f"D{i}"].alignment = Alignment(horizontal="right")

# 請求書番号と発行日を定義
invoice_number = 1  # 整数として請求書番号を定義

# ゼロ埋めで4桁形式にフォーマット
formatted_invoice_number = f"{invoice_number:04}"
issue_date = datetime.now().strftime("%Y/%m/%d")  # 現在の日付をフォーマット

# 請求書番号と日付をExcelシートに記載（2行目の右上に配置）
ws["F4"] = "No."
ws["G4"] = formatted_invoice_number
ws["F4"].alignment = Alignment(horizontal="left")  # 左寄せ
ws["G4"].alignment = Alignment(horizontal="left")  # 左寄せ

ws["F5"] = "日付"
ws["G5"] = issue_date
ws["F5"].alignment = Alignment(horizontal="left")  # 左寄せ
ws["G5"].alignment = Alignment(horizontal="left")  # 左寄せ


# 列幅の調整
ws.column_dimensions["A"].width = 20
ws.column_dimensions["B"].width = 10
ws.column_dimensions["C"].width = 15
ws.column_dimensions["D"].width = 15

# Excelファイルを保存
wb.save(file_name)
print(f"請求書が作成されました: {file_name}")

請求書が作成されました: 請求書_20250413.xlsx
